# Dummy models

To construct an  `InterpolationModel`, or one of its subclasses (such as `ARTModel`), one must choose which degrees of freedom and displacements to consider. Displaced structures must be built, their polarizabilities must be calculated, and finally these polarizabilities must be fed into an appropriate polarizability model. 

There are infinite number of ways to express a system's degrees of freedom; ultimately, this choice depends on user preference as well as the research question at hand. However, before carrying out expensive polarizability calculations, we want to be sure that our DOFs are, at the very lease, **valid**. 

For this purpose, we use dummy models. Essentially, we build up a polarizability model the same way we always have, except we instruct the model to ignore all polarizabilities. Internally, the model generates dummy polarizabilities. Although a dummy model cannot be used to predict polarizabilities (and therefore cannot be used to compute Raman spectra), it gives us a chance to perform a "dry run" before carrying out any first principles calculations.

In this tutorial, show how to use dummy models to ensure that all DOFs are properly specified.

We'll import everything we'll need up front, as well add some customizations for matplotlib.

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib_inline

import ramannoodle.io.vasp as vasp_io
from ramannoodle.polarizability.art import ARTModel
from ramannoodle.structure import symmetry_utils
from ramannoodle.spectrum.spectrum_utils import convolve_spectrum

matplotlib_inline.backend_inline.set_matplotlib_formats('png')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams["mathtext.default"] = 'regular'
plt.rcParams['axes.linewidth'] = 0.5
plt.rcParams['xtick.major.width'] = 0.5
plt.rcParams['xtick.minor.width'] = 0.5
plt.rcParams['lines.linewidth'] = 1.5

Our final goal is to calculate TiO2's Raman spectrum using data available in `tests/data/TiO2`. We will be basing this spectrum on frozen phonon calculations and use `ARTModel` to estimate phonon polarizabilities.

In [2]:
data_dir = "../../../test/data/TiO2"
outcar = f"{data_dir}/phonons_OUTCAR"
ref_structure = vasp_io.outcar.read_ref_structure(outcar)


Now that we have the reference structure, we can build the polarizability model. We do this the usual way, but we specify `is_dummy_model = True`. We also can use any value for `equilibrium_polarizability`, as this parameter is simply ignored.

In [3]:
_, equilibrium_polarizability = vasp_io.outcar.read_positions_and_polarizability(
   f"{data_dir}/ref_eps_OUTCAR"
)
model = ARTModel(ref_structure=ref_structure, 
                 equilibrium_polarizability = np.zeros((3,3)), # This value is completely ignored.
                 is_dummy_model=True)               
model

╭──────────────┬──────────────┬─────────────┬────────────────────╮
│   Atom index │ Directions   │ Specified   │   Equivalent atoms │
├──────────────┼──────────────┼─────────────┼────────────────────┤
│            0 │              │ 0/3         │                 35 │
│           36 │              │ 0/3         │                 71 │
╰──────────────┴──────────────┴─────────────┴────────────────────╯
  ATTENTION: this is a dummy model. 

Note that the `__repr__` string indicates that this is a dummy model.

In [4]:
# Now we can add the outcars. Because we have a dummy model, ramannoodle will not look for polarizabilities in these files. Therefore, we could use OUTCARs from other calculations or (more commonly) use POSCARs.
model.add_art_from_files(
    [f"{data_dir}/Ti5_0.1x_eps_OUTCAR"], file_format = 'outcar'
  )
model.add_art_from_files(
    [f"{data_dir}/Ti5_0.1z_eps_OUTCAR"],file_format = 'outcar'
  )
model.add_art_from_files(
    [f"{data_dir}/O43_0.1z_eps_OUTCAR", f"{data_dir}/O43_m0.1z_eps_OUTCAR"], 
    file_format="outcar"
)
model.add_art_from_files(
    [f"{data_dir}/O43_0.1x_eps_OUTCAR"], file_format = 'outcar'
)
model.add_art_from_files([f"{data_dir}/O43_0.1y_eps_OUTCAR"],file_format = 'outcar')
model

╭──────────────┬─────────────────────────────────────────────┬─────────────┬────────────────────╮
│   Atom index │ Directions                                  │ Specified   │   Equivalent atoms │
├──────────────┼─────────────────────────────────────────────┼─────────────┼────────────────────┤
│            0 │ [-1. -0. +0.], [-0. -1. -0.], [-0. -0. +1.] │ 3/3         │                 35 │
│           36 │ [+0. +0. +1.], [+1. +0. -0.], [+0. +1. -0.] │ 3/3         │                 71 │
╰──────────────┴─────────────────────────────────────────────┴─────────────┴────────────────────╯
  ATTENTION: this is a dummy model. 

From the "Specified" column, we see that all degrees of freedom have been specified. Clearly, our code is working and we chose our DOFs wisely. Nice work! Now what will happen if we try to compute a Raman spectrum with this dummy model?

In [5]:
# Compute and plot spectrum
phonons = vasp_io.outcar.read_phonons(f"{data_dir}/phonons_OUTCAR")
spectrum = phonons.get_raman_spectrum(model)
wavenumbers, total_intensities = spectrum.measure(laser_correction = True, 
                                           laser_wavelength = 532, 
                                           bose_einstein_correction = True, 
                                           temperature = 300)
wavenumbers, total_intensities = convolve_spectrum(wavenumbers, total_intensities)
fig = plt.figure()
axis = fig.add_subplot(111)
axis.plot(wavenumbers, total_intensities)

UsageError: dummy model cannot calculate polarizabilities

Since dummy models contain no polarizability information, they cannot be used to compute spectra! 